In [1]:
import numpy as np
import h5py
from scipy.io import loadmat

In [2]:
def Patch(data,height_index,width_index,PATCH_SIZE):   # PATCH_SIZE为一个patch（边长-1）的一半    data维度(H,W,C)
    height_slice = slice(height_index-PATCH_SIZE, height_index+PATCH_SIZE)
    width_slice = slice(width_index-PATCH_SIZE, width_index+PATCH_SIZE)
    # 由height_index和width_index定位patch中心像素
    patch = data[height_slice, width_slice,:]
    patch = patch.reshape(-1,patch.shape[0]*patch.shape[1]*patch.shape[2])
    # print(patch.shape)                  #为一行  (1, 243) 243 = 9*9*3
    return patch

In [3]:
img = loadmat('../../dataset/1_Houston/Houston.mat')['Houston']
print(img.shape)
gt = loadmat('../../dataset/1_Houston/Houston_gt.mat')['Houston_gt']
print(gt.shape) 
'''
(349, 1905, 144)
(349, 1905)
'''

(349, 1905, 144)
(349, 1905)


'\n(349, 1905, 144)\n(349, 1905)\n'

In [4]:
dict_k = {}
for i in range(gt.shape[0]):
    for j in range(gt.shape[1]):
        if gt[i][j] in range(0, 16):
            if gt[i][j] not in dict_k:
                dict_k[gt[i][j]] = 0
            dict_k[gt[i][j]] += 1

print(dict_k)  

'''
{0: 649816, 9: 1252, 1: 1251, 11: 1235, 2: 1254, 7: 1268, 4: 1244, 10: 1227, 8: 1244, 12: 1233, 5: 1242, 6: 325, 13: 469, 14: 428, 15: 660, 3: 697}
'''

{0: 649816, 9: 1252, 1: 1251, 11: 1235, 2: 1254, 7: 1268, 4: 1244, 10: 1227, 8: 1244, 12: 1233, 5: 1242, 6: 325, 13: 469, 14: 428, 15: 660, 3: 697}


'\n{0: 649816, 9: 1252, 1: 1251, 11: 1235, 2: 1254, 7: 1268, 4: 1244, 10: 1227, 8: 1244, 12: 1233, 5: 1242, 6: 325, 13: 469, 14: 428, 15: 660, 3: 697}\n'

In [5]:
'''
波段处理
'''
bands_select = [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 138, 139, 140, 141, 143, 144]
for i in range(len(bands_select)):
    bands_select[i] = bands_select[i] - 1 
# print(bands_select)

'''
波段选择
'''
img = img[:, :, bands_select]
print(img.shape)

'''
图像归一化
'''
img = ( img * 1.0 - img.min() ) / ( img.max() - img.min() )
type(img)

(349, 1905, 100)


numpy.ndarray

In [6]:
[m, n, b] = img.shape
label_num = gt.max()  #最大为9，即除0外包括9类
print(label_num)
PATCH_SIZE = 14

15


In [7]:
# padding the hyperspectral images
img_temp = np.zeros((m + 2 * PATCH_SIZE, n + 2 * PATCH_SIZE, b), dtype=np.float32)
img_temp[PATCH_SIZE:(m + PATCH_SIZE), PATCH_SIZE:(n + PATCH_SIZE), :] = img[:, :, :]

for i in range(PATCH_SIZE):
    img_temp[i, :, :] = img_temp[2 * PATCH_SIZE - i, :, :]
    img_temp[m + PATCH_SIZE + i, :, :] = img_temp[m + PATCH_SIZE - i - 2, :, :]

for i in range(PATCH_SIZE):
    img_temp[:, i, :] = img_temp[:, 2 * PATCH_SIZE - i, :]
    img_temp[:, n + PATCH_SIZE + i, :] = img_temp[:, n + PATCH_SIZE  - i - 2, :]

img = img_temp

gt_temp = np.zeros((m + 2 * PATCH_SIZE, n + 2 * PATCH_SIZE), dtype=np.int8)
gt_temp[PATCH_SIZE:(m + PATCH_SIZE), PATCH_SIZE:(n + PATCH_SIZE)] = gt[:, :]
gt = gt_temp


In [8]:
print(img.shape, gt.shape)

(377, 1933, 100) (377, 1933)


In [9]:
[m, n, b] = img.shape

In [10]:
data = []
label = []
count = 0 #统计有多少个中心像素类别不为0的patch

In [11]:
for i in range(PATCH_SIZE, m - PATCH_SIZE): # PATCH_SIZE, 50, 100, 150, 200, 250, 300, 330, m - PATCH_SIZE
    for j in range(PATCH_SIZE, n - PATCH_SIZE):
        if gt[i, j] == 0:
            continue
        else:
            count += 1
            temp_data = Patch(img, i, j, PATCH_SIZE)
            temp_label = gt[i, j] - 1

            data.append(temp_data)  # 每一行表示一个patch
            label.append(temp_label)
print(count)  #15029

15029


In [12]:
data = np.array(data)
print(data.shape)  # (15029, 1, 78400)

data = np.squeeze(data)
print("squeeze : ", data.shape)  # squeeze :  (15029, 78400)

label = np.array(label)
print(label.shape)  #(15029,)

label = np.squeeze(label)
print("squeeze : ", label.shape)  # squeeze :  (15029,)

(15029, 1, 78400)
squeeze :  (15029, 78400)
(15029,)
squeeze :  (15029,)


In [13]:
print(type(label))
hs_dict = {}
hs_dict = hs_dict.fromkeys([i for i in range(label_num)], 0)
print(hs_dict)

for i in label:
    if i in hs_dict:
        hs_dict[i] += 1
print(hs_dict)
hs_label_sum = sum(hs_dict.values())
print(hs_label_sum)

<class 'numpy.ndarray'>
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0}
{0: 1251, 1: 1254, 2: 697, 3: 1244, 4: 1242, 5: 325, 6: 1268, 7: 1244, 8: 1252, 9: 1227, 10: 1235, 11: 1233, 12: 469, 13: 428, 14: 660}
15029


In [13]:
indices = np.arange(data.shape[0])  # list [0,.....,42775]
shuffled_indices = np.random.permutation(indices)
images = data[shuffled_indices]
labels = label[shuffled_indices]  # 打乱顺序

In [14]:
y = labels  # 布尔索引
# y 为一个list (42776,)  [0 1 6 ... 0 7 1]

n_classes = y.max() + 1  # y.max() = 8  n_classes代表类别数为9
t_labeled = []

In [15]:
for c in range(n_classes):  # 对于每一类     每类的pixel数量：6631 18649 2099 3064 1345 5029 1330 3682 947
    #if dict_sample[str(c+1)]<200:
    #    pass
    #else:
    i = indices[y == c][:200]
    t_labeled += list(i)  # 列表中元素增加

In [16]:
t_images = images[t_labeled]
print('t_images', t_images.shape)
t_labels = labels[t_labeled]
print('t_labels', t_labels.shape)

t_images (3000, 78400)
t_labels (3000,)


In [17]:
f = h5py.File(r'../../h5dataset_ica_bandselect_200/HS-' + str(PATCH_SIZE * 2) + '-' + str(PATCH_SIZE * 2) + '-100.h5', 'w')  # 每类200个
f['data'] = t_images
f['label'] = t_labels
f.close()
print('success')

success


In [18]:
f = h5py.File(r'../../h5dataset_ica_bandselect/HS-' + str(PATCH_SIZE * 2) + '-' + str(PATCH_SIZE * 2) + '-100.h5', 'w')  # 每类200个
f['data'] = images
f['label'] = labels
f.close()
print('success')

success


In [19]:
#HDF5的读取
f = h5py.File('../../h5dataset_ica_bandselect/HS-28-28-100.h5','r')   #打开h5文件  # 可以查看所有的主键  
# print(type(f))

print([key for key in f.keys()])

# print('first, we get values of x:', f['data'][:])
# print('then, we get values of y:', f['label'][:])
print(f['data'][:].shape)
print(f['label'][:].shape)

['data', 'label']
(15029, 78400)
(15029,)


In [20]:
#HDF5的读取
f = h5py.File('../../h5dataset_ica_bandselect_200/HS-28-28-100.h5','r')   #打开h5文件  # 可以查看所有的主键  
# print(type(f))

print([key for key in f.keys()])

# print('first, we get values of x:', f['data'][:])
# print('then, we get values of y:', f['label'][:])
print(f['data'][:].shape)
print(f['label'][:].shape)

['data', 'label']
(3000, 78400)
(3000,)


In [21]:
#HDF5的读取
f = h5py.File('../../h5dataset/HS-28-28-100.h5','r')   #打开h5文件  # 可以查看所有的主键  
# print(type(f))

print([key for key in f.keys()])

# print('first, we get values of x:', f['data'][:])
# print('then, we get values of y:', f['label'][:])
print(f['data'][:].shape)
print(f['label'][:].shape)

['data', 'label']
(3000, 78400)
(3000,)
